# Imports

In [69]:
# General

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Hyperparameter tuning
import optuna

# Models
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# Data processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

# Data preparation

In [70]:
# Data input

try:
    data = pd.read_csv('../data/train.csv')
    data_to_predict = pd.read_csv('../data/test.csv')
    data_ccrisk = pd.read_csv('../data/credit_risk_dataset.csv')

except:
    data = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
    data_to_predict = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
    data_ccrisk = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')


data_ccrisk.dropna(inplace=True)
data_ccrisk.drop_duplicates(inplace=True)

X_old = data.drop(['loan_status'], axis=1)
y_old = data['loan_status']

# Merge the dataframes
data_no_id = data.drop(['id'], axis=1)
merged_data = pd.concat([data_no_id, data_ccrisk], ignore_index=True)

X = merged_data.drop(['loan_status'], axis=1)
y = merged_data['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.2, random_state=2024)

In [73]:
from imblearn.over_sampling import SMOTENC

categorical_columns = X.select_dtypes(include=['object']).columns

sm = SMOTENC(sampling_strategy=0.3, categorical_features=[X_train.columns.get_loc(col) for col in categorical_columns], random_state=2024)

X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [74]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

# Data processing
categorical_columns = X.select_dtypes(include=['object']).columns

categorical_ordinal = ['loan_grade']
categorical_onehot = categorical_columns.drop(categorical_ordinal)

log_columns = ['person_age', 'person_income', 'person_emp_length', 'loan_amnt']
log_columns = log_columns

numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
numerical_columns = numerical_columns.drop(log_columns)

def log_transform():
	return FunctionTransformer(np.log1p, validate=True)

preprocessor = ColumnTransformer(
	transformers=[
		('ordinal', OrdinalEncoder(), categorical_ordinal),
		('onehot', OneHotEncoder(), categorical_onehot),
		('scaler', StandardScaler(), numerical_columns),
		('log', log_transform(), log_columns)
	], remainder='passthrough'
	)

preprocessor.fit(X_train_res)

X_train_prep = preprocessor.transform(X_train_res)
X_test_prep = preprocessor.transform(X_test)

# Models

## XGBoost

In [77]:
def xgb_objective(trial):
    # Hyperparameter search space
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'eval_metric': 'auc',  # AUC is the metric
        'objective': 'binary:logistic'  # Use binary logistic, probability outputs
    }

    # Initialize the model
    model = xgb.XGBClassifier(**param)

    # Fit the model
    model.fit(X_train_prep, y_train_res,
              eval_set=[(X_test_prep, y_test)],  
              verbose=False)

    # Predict proba for validation set
    y_pred_prob = model.predict_proba(X_test_prep)[:, 1]

    auc = roc_auc_score(y_test, y_pred_prob)
 
    return auc

# Study object to run the optimization. I want to maximize AUC
xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=50)

print(f"Best trial: {xgb_study.best_trial.params}")

[I 2024-10-12 19:30:40,044] A new study created in memory with name: no-name-a419244e-6b70-446a-abca-d2e02890931a


[I 2024-10-12 19:30:48,466] Trial 0 finished with value: 0.9509918617997795 and parameters: {'max_depth': 6, 'learning_rate': 0.2753359156029285, 'n_estimators': 470, 'subsample': 0.8930516628715627, 'colsample_bytree': 0.549635675574224, 'gamma': 0.3869428290930283, 'lambda': 6.663872006442553e-07, 'alpha': 0.08758235756615641, 'scale_pos_weight': 1.6035776672634539}. Best is trial 0 with value: 0.9509918617997795.
[I 2024-10-12 19:30:53,035] Trial 1 finished with value: 0.936974283385811 and parameters: {'max_depth': 3, 'learning_rate': 0.03228938470800746, 'n_estimators': 310, 'subsample': 0.7238808570425561, 'colsample_bytree': 0.7849660409677419, 'gamma': 0.468314165373791, 'lambda': 0.0013806290894327314, 'alpha': 0.004836910624548605, 'scale_pos_weight': 1.599625055404122}. Best is trial 0 with value: 0.9509918617997795.
[I 2024-10-12 19:30:58,394] Trial 2 finished with value: 0.9565239866581974 and parameters: {'max_depth': 8, 'learning_rate': 0.08196270349822926, 'n_estimators

Best trial: {'max_depth': 6, 'learning_rate': 0.08712961838528847, 'n_estimators': 435, 'subsample': 0.9168468853937433, 'colsample_bytree': 0.7801433420045394, 'gamma': 0.21389666989069075, 'lambda': 7.568525836977652e-05, 'alpha': 0.0005838263233074763, 'scale_pos_weight': 1.005561624866418}


In [78]:
xgb_best_params = xgb_study.best_trial.params
xgb_best_score = xgb_study.best_trial.value
xgb_best_score

0.9578972528897483

prev: 0.9576051262733373

## CatBoostClassifier

In [80]:
from catboost import CatBoostClassifier

def cat_objective(trial):

    # Define the hyperparameter search space

    param = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'iterations': trial.suggest_int('iterations', 200, 1000),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'eval_metric': 'AUC',  
        'logging_level': 'Silent',  # Disable CatBoost output
        'task_type': 'CPU',  # Try using GPU
        'use_best_model': True
    }

    # Initialize the CatBoost model with the trial's parameters
    model = CatBoostClassifier(**param)

    # Fit the model
    model.fit(X_train_prep, y_train_res,
              eval_set=[(X_test_prep, y_test)],
              early_stopping_rounds=20,  # Early stopping to prevent overfitting
              verbose=False)

    # Get predictions and calculate AUC score
    y_pred_prob = model.predict_proba(X_test_prep)[:, 1]  # Probabilities for class 1

    auc = roc_auc_score(y_test, y_pred_prob)

    return auc
    
cat_study = optuna.create_study(direction='maximize')  # We want to maximize AUC
cat_study.optimize(cat_objective, n_trials=50)

# Output the best trial
print(f"Best trial: {cat_study.best_trial.params}")

[I 2024-10-12 19:40:42,509] A new study created in memory with name: no-name-d2997b63-b960-42da-8418-d17026932fee
[I 2024-10-12 19:40:49,576] Trial 0 finished with value: 0.9516252389061267 and parameters: {'depth': 7, 'learning_rate': 0.14923111169375491, 'iterations': 220, 'l2_leaf_reg': 9.175977789900557e-06, 'border_count': 235, 'bagging_temperature': 0.2819519131072058, 'random_strength': 0.38014451113574343, 'scale_pos_weight': 2.0650612435134765}. Best is trial 0 with value: 0.9516252389061267.
[I 2024-10-12 19:41:04,769] Trial 1 finished with value: 0.9449467364130162 and parameters: {'depth': 7, 'learning_rate': 0.05129892236922186, 'iterations': 853, 'l2_leaf_reg': 8.211830203332446e-06, 'border_count': 37, 'bagging_temperature': 0.976816357191858, 'random_strength': 0.17116699653918355, 'scale_pos_weight': 2.9830174834750443}. Best is trial 0 with value: 0.9516252389061267.
[I 2024-10-12 19:41:13,082] Trial 2 finished with value: 0.9482456963745374 and parameters: {'depth': 

Best trial: {'depth': 4, 'learning_rate': 0.29883221631968115, 'iterations': 490, 'l2_leaf_reg': 0.3645825533719849, 'border_count': 231, 'bagging_temperature': 0.7809357127963482, 'random_strength': 0.6205304341507932, 'scale_pos_weight': 1.9590564128414854}


In [81]:
cat_best_params = cat_study.best_trial.params
cat_best_score = cat_study.best_trial.value
cat_best_score

0.9585876155837739

prev: 0.9601213839193278

## LightGBM

In [82]:
# Objective function for LightGBM

def lgb_objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }

    lgb_model = lgb.LGBMClassifier(**param)
  

    # Train the model
    lgb_model.fit(X_train_prep, y_train_res,
                  eval_set=[(X_test_prep, y_test)],
                  eval_metric='auc'
                  )

    # Predict and evaluate AUC
    y_pred_prob = lgb_model.predict_proba(X_test_prep)[:, 1]
    auc = roc_auc_score(y_test, y_pred_prob)

    return auc

# Optimize the objective function
lgb_study = optuna.create_study(direction='maximize')
lgb_study.optimize(lgb_objective, n_trials=50)

# Best parameters and AUC
print("Best LGBM Params: ", lgb_study.best_trial.params)
print("Best AUC for LGBM: ", lgb_study.best_value)

[I 2024-10-12 19:47:59,692] A new study created in memory with name: no-name-2288c113-9bce-4922-9ca4-d5444d67e626


[LightGBM] [Info] Number of positive: 17425, number of negative: 58086
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010680 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1328
[LightGBM] [Info] Number of data points in the train set: 75511, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.230761 -> initscore=-1.204019
[LightGBM] [Info] Start training from score -1.204019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-10-12 19:48:08,093] Trial 0 finished with value: 0.9532816689263568 and parameters: {'learning_rate': 0.14660732960628925, 'num_leaves': 122, 'max_depth': 15, 'min_child_samples': 67, 'min_child_weight': 1.9934250980471022, 'subsample': 0.595440920984583, 'colsample_bytree': 0.5226370318101002, 'n_estimators': 695}. Best is trial 0 with value: 0.9532816689263568.


[LightGBM] [Info] Number of positive: 17425, number of negative: 58086
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010882 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1328
[LightGBM] [Info] Number of data points in the train set: 75511, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.230761 -> initscore=-1.204019
[LightGBM] [Info] Start training from score -1.204019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-10-12 19:48:11,215] Trial 1 finished with value: 0.9598445900135015 and parameters: {'learning_rate': 0.21616636469118897, 'num_leaves': 88, 'max_depth': 3, 'min_child_samples': 42, 'min_child_weight': 4.3746928508639265, 'subsample': 0.6870420272315677, 'colsample_bytree': 0.7644315655489906, 'n_estimators': 538}. Best is trial 1 with value: 0.9598445900135015.


...

[LightGBM] [Info] Number of positive: 17425, number of negative: 58086
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005304 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1328
[LightGBM] [Info] Number of data points in the train set: 75511, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.230761 -> initscore=-1.204019
[LightGBM] [Info] Start training from score -1.204019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-10-12 19:52:32,889] Trial 48 finished with value: 0.9591129533201724 and parameters: {'learning_rate': 0.040054877681874164, 'num_leaves': 80, 'max_depth': 6, 'min_child_samples': 65, 'min_child_weight': 5.915853976004982, 'subsample': 0.8971753640297574, 'colsample_bytree': 0.7533253483685995, 'n_estimators': 511}. Best is trial 22 with value: 0.9609191584669619.


[LightGBM] [Info] Number of positive: 17425, number of negative: 58086
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1328
[LightGBM] [Info] Number of data points in the train set: 75511, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.230761 -> initscore=-1.204019
[LightGBM] [Info] Start training from score -1.204019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-10-12 19:52:37,419] Trial 49 finished with value: 0.9511333844944644 and parameters: {'learning_rate': 0.01937644423109847, 'num_leaves': 117, 'max_depth': 5, 'min_child_samples': 84, 'min_child_weight': 5.074722678393634, 'subsample': 0.9747327043737702, 'colsample_bytree': 0.6123245916882544, 'n_estimators': 575}. Best is trial 22 with value: 0.9609191584669619.


Best LGBM Params:  {'learning_rate': 0.08183082759159926, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 72, 'min_child_weight': 6.58015466678275, 'subsample': 0.9264934033935981, 'colsample_bytree': 0.507869285171017, 'n_estimators': 632}
Best AUC for LGBM:  0.9609191584669619


In [83]:
lgb_best_params = lgb_study.best_trial.params
lgb_best_score = lgb_study.best_value
lgb_best_score

0.9609191584669619

prev: 0.9607718460460408

## ExtraTreesClassifier

In [84]:
from sklearn.ensemble import ExtraTreesClassifier

def extratrees_objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0)
    }

    model = ExtraTreesClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train_res, cv=3, scoring='roc_auc').mean()

    return score

extratrees_study = optuna.create_study(direction='maximize')
extratrees_study.optimize(extratrees_objective, n_trials=50, timeout=600)

print('Best parameters for ExtraTrees:', extratrees_study.best_params)

[I 2024-10-12 19:52:37,455] A new study created in memory with name: no-name-3cf88c50-ee8b-4551-ace4-0e9fdd49d23f
[I 2024-10-12 19:53:39,265] Trial 0 finished with value: 0.9371687502088122 and parameters: {'n_estimators': 400, 'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 7, 'max_features': 0.3867833531287491}. Best is trial 0 with value: 0.9371687502088122.
[I 2024-10-12 19:54:22,151] Trial 1 finished with value: 0.9432823286929178 and parameters: {'n_estimators': 193, 'max_depth': 18, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 0.4548813453061207}. Best is trial 1 with value: 0.9432823286929178.
[I 2024-10-12 19:54:42,847] Trial 2 finished with value: 0.9108602505621208 and parameters: {'n_estimators': 145, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 0.7864539322777361}. Best is trial 1 with value: 0.9432823286929178.
[I 2024-10-12 19:56:03,913] Trial 3 finished with value: 0.9385142682873622 and parameters: {'n_es

Best parameters for ExtraTrees: {'n_estimators': 373, 'max_depth': 17, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 0.6716623956987676}


In [85]:
extratrees_best_params = extratrees_study.best_params
extratrees_best_score = extratrees_study.best_value
extratrees_best_score

0.9454722065252454

Prev: 0.9298212453316017

## LogisticRegression

In [86]:
from sklearn.linear_model import LogisticRegression

def logreg_objective(trial):
    
    param = {
        'penalty': 'elasticnet',
        'solver' : 'saga',
        'C': trial.suggest_float('C', 1e-4, 1e2, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', 0, 1)
    }

    model = LogisticRegression(**param, max_iter=1000)
    score = cross_val_score(model, X_train_prep, y_train_res, cv=3, scoring='roc_auc').mean()
    return score

logreg_study = optuna.create_study(direction='maximize')
logreg_study.optimize(logreg_objective, n_trials=50, timeout=600)
print('Best parameters for Logistic Regression:', logreg_study.best_params)

[I 2024-10-12 20:03:44,828] A new study created in memory with name: no-name-6c70cb0f-4583-452c-8958-082284b5c602
[I 2024-10-12 20:04:07,519] Trial 0 finished with value: 0.8949578149433436 and parameters: {'C': 60.539495455962374, 'l1_ratio': 0.9390717209437502}. Best is trial 0 with value: 0.8949578149433436.
[I 2024-10-12 20:04:27,531] Trial 1 finished with value: 0.8949582417887815 and parameters: {'C': 20.40603404997764, 'l1_ratio': 0.977805112586921}. Best is trial 1 with value: 0.8949582417887815.
[I 2024-10-12 20:04:37,758] Trial 2 finished with value: 0.8796284810536301 and parameters: {'C': 0.0007736395458854829, 'l1_ratio': 0.3291803063019507}. Best is trial 1 with value: 0.8949582417887815.
[I 2024-10-12 20:05:01,707] Trial 3 finished with value: 0.8949579779890876 and parameters: {'C': 60.74454912443483, 'l1_ratio': 0.24245475272025174}. Best is trial 1 with value: 0.8949582417887815.
c:\Users\Denis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_m

Best parameters for Logistic Regression: {'C': 0.034635673794836994, 'l1_ratio': 0.667050998518569}


In [87]:
logreg_best_params = logreg_study.best_params
logreg_best_params['penalty'] = 'elasticnet'
logreg_best_params['solver'] = 'saga'
logreg_best_score = logreg_study.best_value
logreg_best_score

0.8951398196537403

prev: 0.8814172446287084

## HistGradientBoostingClassifier

In [88]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

def histgb_objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 50),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 50),
        'l2_regularization': trial.suggest_float('l2_regularization', 1e-5, 1.0, log=True)
    }

    model = HistGradientBoostingClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train_res, cv=3, scoring='roc_auc').mean()

    return score

histgb_study = optuna.create_study(direction='maximize')
histgb_study.optimize(histgb_objective, n_trials=50, timeout=600)
print('Best parameters for HistGradientBoosting:', histgb_study.best_params)

[I 2024-10-12 20:14:31,639] A new study created in memory with name: no-name-a52446d3-06ca-4ec4-995b-c11e8809ba86
[I 2024-10-12 20:14:58,204] Trial 0 finished with value: 0.9527105434383119 and parameters: {'learning_rate': 0.015891334732469724, 'max_iter': 392, 'max_depth': 9, 'min_samples_leaf': 31, 'max_leaf_nodes': 21, 'l2_regularization': 0.07851274814154947}. Best is trial 0 with value: 0.9527105434383119.
[I 2024-10-12 20:15:23,420] Trial 1 finished with value: 0.9594885638898828 and parameters: {'learning_rate': 0.038004830266586756, 'max_iter': 410, 'max_depth': 17, 'min_samples_leaf': 12, 'max_leaf_nodes': 44, 'l2_regularization': 0.02165795937237716}. Best is trial 1 with value: 0.9594885638898828.
[I 2024-10-12 20:15:37,556] Trial 2 finished with value: 0.9562189548392435 and parameters: {'learning_rate': 0.036554249284055394, 'max_iter': 217, 'max_depth': 10, 'min_samples_leaf': 28, 'max_leaf_nodes': 27, 'l2_regularization': 5.864025123950696e-05}. Best is trial 1 with val

Best parameters for HistGradientBoosting: {'learning_rate': 0.09941268072767732, 'max_iter': 451, 'max_depth': 20, 'min_samples_leaf': 19, 'max_leaf_nodes': 34, 'l2_regularization': 0.00045378231647806653}


In [89]:
histgb_best_params = histgb_study.best_trial.params
histgb_best_score = histgb_study.best_trial.value
histgb_best_score

0.9601709868096956

Prev: 0.9519987809872763

## KNN

In [90]:
from sklearn.neighbors import KNeighborsClassifier

def knn_objective(trial):
    param = {
        'n_neighbors': trial.suggest_int('n_neighbors', 3, 15),
        'leaf_size': trial.suggest_int('leaf_size', 20, 50),
        'p': trial.suggest_categorical('p', [1, 2]),  # Minkowski distance parameter
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance'])
    }

    model = KNeighborsClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train_res, cv=3, scoring='roc_auc').mean()

    return score

knn_study = optuna.create_study(direction='maximize')
knn_study.optimize(knn_objective, n_trials=50, timeout=600)
print('Best parameters for KNN:', knn_study.best_params)
print('Best score for KNN:', knn_study.best_value)

[I 2024-10-12 20:24:38,186] A new study created in memory with name: no-name-34d1daae-4bb1-494a-a072-e1c104fe4712
[I 2024-10-12 20:25:36,573] Trial 0 finished with value: 0.9133490596201589 and parameters: {'n_neighbors': 11, 'leaf_size': 46, 'p': 1, 'weights': 'uniform'}. Best is trial 0 with value: 0.9133490596201589.
[I 2024-10-12 20:26:30,773] Trial 1 finished with value: 0.910168120147794 and parameters: {'n_neighbors': 8, 'leaf_size': 35, 'p': 1, 'weights': 'distance'}. Best is trial 0 with value: 0.9133490596201589.
[I 2024-10-12 20:26:46,738] Trial 2 finished with value: 0.9020045107791387 and parameters: {'n_neighbors': 8, 'leaf_size': 28, 'p': 2, 'weights': 'uniform'}. Best is trial 0 with value: 0.9133490596201589.
[I 2024-10-12 20:27:42,534] Trial 3 finished with value: 0.910168120147794 and parameters: {'n_neighbors': 8, 'leaf_size': 38, 'p': 1, 'weights': 'distance'}. Best is trial 0 with value: 0.9133490596201589.
[I 2024-10-12 20:27:59,012] Trial 4 finished with value: 

Best parameters for KNN: {'n_neighbors': 13, 'leaf_size': 24, 'p': 1, 'weights': 'distance'}
Best score for KNN: 0.9182720698807708


In [91]:
knn_best_params = knn_study.best_trial.params
knn_best_score = knn_study.best_trial.value
knn_best_score

0.9182720698807708

Prev: 0.9003336325395043

# Stacking

In [92]:
from sklearn.ensemble import StackingClassifier

# Init base models

xgb_model = xgb.XGBClassifier(**xgb_best_params)
cat_model = CatBoostClassifier(**cat_best_params)
lgb_model = lgb.LGBMClassifier(**lgb_best_params)
extratrees_model = ExtraTreesClassifier(**extratrees_best_params)
logreg_model = LogisticRegression(**logreg_best_params)
histgb_model = HistGradientBoostingClassifier(**histgb_best_params)
knn_model = KNeighborsClassifier(**knn_best_params)

# Base and meta models
base_estimators = [
	('xgb', xgb_model),
	('cat', cat_model),
	('lgb', lgb_model),
	('extratrees', extratrees_model),
	('logreg', logreg_model),
	('histgb', histgb_model),
	('knn', knn_model)
]

meta_model = LogisticRegression()

# Stack

stack_model = StackingClassifier(estimators=base_estimators, final_estimator=meta_model, cv=5, n_jobs=-1)

stack_model.fit(X_train_prep, y_train_res)

StackingClassifier(cv=5,
                   estimators=[('xgb',
                                XGBClassifier(alpha=0.0005838263233074763,
                                              base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.7801433420045394,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None,
                                              gamma=0.21389666989069075,
                                              grow_policy=No...
                                                   l1_ratio=0.667050998518569,
                                                   penalty='elasticnet',
                                                   solver='saga')),
                               ('histgb',
                                HistGradientBoostingClassifier(l2_regularization=0.00045378231647806653,
                                                               learning_rate=0.09941268072767732,
                                                               max_depth=20,
                                                               max_iter=451,
                                                               max_leaf_nodes=34,
                                                               min_samples_leaf=19)),
                               ('knn',
                                KNeighborsClassifier(leaf_size=24,
                                                     n_neighbors=13, p=1,
                                                     weights='distance'))],
                   final_estimator=LogisticRegression(), n_jobs=-1)

In [93]:
print('Stacking Classifier ROC AUC:', roc_auc_score(y_test, stack_model.predict_proba(X_test_prep)[:, 1]))

Stacking Classifier ROC AUC: 0.9607975623760714


Prev: 0.9614965682196932

In [ ]:
# Choose different base models

base_estimators = [
	('xgb', xgb_model),
	('cat', cat_model),
	('lgb', lgb_model),
	('histgb', histgb_model),
	('knn', knn_model)

]

meta_model = lgb.LGBMClassifier()

# Stack
stack_model_2 = StackingClassifier(estimators=base_estimators, final_estimator=meta_model, cv=3, n_jobs=-1)

stack_model_2.fit(X_train_prep, y_train_res)

print('Stacking Classifier ROC AUC:', roc_auc_score(y_test, stack_model_2.predict_proba(X_test_prep)[:, 1]))

# Submission

In [65]:
smote = SMOTENC(sampling_strategy=0.3, categorical_features=[X.columns.get_loc(col) for col in categorical_columns], random_state=2024)
X_res, y_res = smote.fit_resample(X, y)

X_train_all = preprocessor.transform(X_res)

stack_model.fit(X_train_all, y_res)

StackingClassifier(cv=5,
                   estimators=[('xgb',
                                XGBClassifier(alpha=1.7764720274807023e-05,
                                              base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.7202079398862329,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None,
                                              gamma=0.20887266737138027,
                                              grow_policy=N...
                                                   l1_ratio=0.890464449999595,
                                                   penalty='elasticnet',
                                                   solver='saga')),
                               ('histgb',
                                HistGradientBoostingClassifier(l2_regularization=2.024705049363233e-05,
                                                               learning_rate=0.0973302436339639,
                                                               max_depth=20,
                                                               max_iter=350,
                                                               max_leaf_nodes=40,
                                                               min_samples_leaf=21)),
                               ('knn',
                                KNeighborsClassifier(leaf_size=34,
                                                     n_neighbors=10, p=1,
                                                     weights='distance'))],
                   final_estimator=LogisticRegression(), n_jobs=-1)

In [67]:
import joblib

try:
    joblib.dump(stack_model, '../models/stack_model_03_smote.pkl')
except:
    joblib.dump(stack_model, 'stack_model_03_smote.pkl')
    
X_to_pred = preprocessor.transform(data_to_predict)
y_pred_submit = stack_model.predict_proba(X_to_pred)[:, 1]

submission = pd.DataFrame({'id': data_to_predict['id'], 'loan_status': y_pred_submit})
try:
    submission.to_csv('../submissions/stack_model_03_somte.csv', index=False)
except:
    submission.to_csv('stack_model_03_somte.csv', index=False)